In [26]:
import random
import numpy as np

class Game2048:
    def __init__(self):
        """Initialize the game board with two random tiles."""
        self.board = np.zeros((4, 4), dtype=int)
        self.add_new_tile()
        self.add_new_tile()

    def add_new_tile(self):
        """Adds a new tile (2 or 4) to a random empty spot on the board."""
        empty_cells = [(i, j) for i in range(4) for j in range(4) if self.board[i, j] == 0]
        if empty_cells:
            i, j = random.choice(empty_cells)
            self.board[i, j] = 2 if random.random() < 0.9 else 4

    def slide_and_merge(self, row):
        """Slides non-zero elements to the left and merges equal adjacent elements."""
        non_zero = row[row != 0]
        new_row = []
        skip = False
        for i in range(len(non_zero)):
            if skip:
                skip = False
                continue
            if i < len(non_zero) - 1 and non_zero[i] == non_zero[i + 1]:
                new_row.append(non_zero[i] * 2)
                skip = True
            else:
                new_row.append(non_zero[i])
        return np.array(new_row + [0] * (4 - len(new_row)))

    def move(self, direction):
        """Applies a move (0: left, 1: up, 2: right, 3: down)."""
        rotated_board = np.rot90(self.board, k=direction)
        for i in range(4):
            rotated_board[i] = self.slide_and_merge(rotated_board[i])
        self.board = np.rot90(rotated_board, k=(-direction % 4))

    def is_game_over(self):
        """Checks if there are no valid moves left."""
        if 0 in self.board:
            return False
        for i in range(4):
            for j in range(4):
                if i < 3 and self.board[i, j] == self.board[i + 1, j]:
                    return False
                if j < 3 and self.board[i, j] == self.board[i, j + 1]:
                    return False
        return True

    def get_possible_moves(self):
        """Returns a list of valid moves (0: left, 1: up, 2: right, 3: down)."""
        moves = []
        for direction in range(4):
            temp_board = self.board.copy()
            rotated_board = np.rot90(temp_board, k=direction)
            new_board = np.array([self.slide_and_merge(row) for row in rotated_board])
            if not np.array_equal(new_board, rotated_board):
                moves.append(direction)
        return moves

    def get_score(self):
        """Returns the highest tile value on the board."""
        return np.max(self.board)

    def visualize_board(self):
        """Displays the current board with grid lines."""
        row_divider = "+" + "+".join(["-" * 6] * 4) + "+"
        print(row_divider)
        for row in self.board:
            row_display = "|".join(f"{value or ' ':^6}" for value in row)
            print(f"|{row_display}|")
            print(row_divider)

class ExpectimaxAI:
    def __init__(self, game):
        self.game = game

    def expectimax(self, board, depth, is_maximizing):
        """Recursive Expectimax algorithm to calculate the best move."""
        if depth == 0 or self.is_terminal(board):
            return self.evaluate(board)

        if is_maximizing:
            # Player's turn: maximize score
            best_score = float('-inf')
            for move in self.game.get_possible_moves():
                new_game = Game2048()
                new_game.board = board.copy()
                new_game.move(move)
                score = self.expectimax(new_game.board, depth - 1, False)
                best_score = max(best_score, score)
            return best_score
        else:
            # Random tile placement: calculate expected score
            empty_cells = [(i, j) for i in range(4) for j in range(4) if board[i, j] == 0]
            if not empty_cells:
                return self.evaluate(board)
            score = 0
            for i, j in empty_cells:
                for tile_value, probability in [(2, 0.9), (4, 0.1)]:
                    new_board = board.copy()
                    new_board[i, j] = tile_value
                    score += probability * self.expectimax(new_board, depth - 1, True)
            return score

    def is_terminal(self, board):
        """Checks if the game is over (no valid moves)."""
        temp_game = Game2048()
        temp_game.board = board.copy()
        return len(temp_game.get_possible_moves()) == 0

    def evaluate(self, board):
        """Heuristic evaluation function for a given board."""
        return np.max(board) + np.sum(board)

    def get_best_move(self, depth=3):
        """Finds the best move by running Expectimax."""
        best_score = float('-inf')
        best_move = None
        for move in self.game.get_possible_moves():
            new_game = Game2048()
            new_game.board = self.game.board.copy()
            new_game.move(move)
            score = self.expectimax(new_game.board, depth - 1, False)
            if score > best_score:
                best_score = score
                best_move = move
        return best_move

# Main Game Loop
game = Game2048()
ai = ExpectimaxAI(game)

while not game.is_game_over():
    print("\nCurrent Board:")
    game.visualize_board()
    # adjust depth of search here
    move = ai.get_best_move(depth=3)
    if move is not None:
        game.move(move)
        game.add_new_tile()
    else:
        break

print("\nGame Over! Final Board:")
game.visualize_board()
print("Max Tile Achieved:", game.get_score())



Current Board:
+------+------+------+------+
|      |  2   |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |  2   |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+

Current Board:
+------+------+------+------+
|      |      |      |  2   |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |  4   |      |      |
+------+------+------+------+

Current Board:
+------+------+------+------+
|      |      |      |  2   |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|  2   |      |      |  4   |
+------+------+------+------+

Current Board:
+------+------+------+------+
|  2   |      |      |      |
+------+------+------+------+
|      |      |      |  2   |
+-----

# **Summary** <br>


#### **Purpose of the Code**
This program implements:
1. **2048 Game Logic**:
   - Includes tile movement, merging, random tile spawning, and game-over conditions.
2. **Expectimax Algorithm**:
   - AI plays optimally by simulating moves and random tile placements.

---

#### **Relation to Game-Playing Algorithms**
1. **Expectimax**:
   - Models both the **player's moves (maximizing)** and the **random tile spawns (averaging)**.
   - Alternates between:
     - **Maximizing node** (player's turn): Simulates all possible moves to find the one with the highest expected score.
     - **Chance node** (random tile placement): Calculates the expected score for all possible random tile positions.

2. **Game-Playing Loop**:
   - The AI uses `get_best_move()` to decide the optimal move based on the `expectimax` search.

3. **Heuristic Evaluation**:
   - The evaluation function (`evaluate`) estimates board quality by summing all tiles and prioritizing the maximum tile.

---

#### **How It Works**
1. **Initialization**:
   - The game starts with two random tiles.
2. **AI Decision-Making**:
   - The AI evaluates moves using `Expectimax` with a depth of 3.
3. **Move Execution**:
   - After the AI decides, the board updates with the selected move and spawns a new tile.
4. **Game Over**:
   - The loop ends when no valid moves remain.

---

# **Game-playing algorithm detailed breakdown** <br>

### **1. What is Expectimax?**
Expectimax is a **decision-making algorithm** designed for games that involve both:
1. **Deterministic actions**: Moves the player can control (e.g., sliding tiles in 2048).
2. **Random events**: Outcomes governed by probability (e.g., new tiles spawning randomly).

It extends the **Minimax algorithm** by replacing the "Minimizing Player" with **Chance Nodes**, which calculate the **expected value** of random events.

---

### **2. How Does Expectimax Work in 2048?**
In 2048, the algorithm alternates between two types of nodes:
1. **Maximizing Nodes**: Represent the AI's turn.
   - The AI evaluates all possible moves (left, right, up, down) and selects the one that maximizes the expected score.

2. **Chance Nodes**: Represent random tile placements.
   - A new tile (either `2` with 90% probability or `4` with 10%) spawns in a random empty cell.
   - The algorithm calculates the expected value of all possible tile placements.

---

### **3. Steps of Expectimax in the Code**

#### **Step 1: Starting the Recursive Search**
The recursion starts with the current board and alternates between:
- **Maximizing player**: AI chooses the best move.
- **Chance node**: Evaluates all random tile spawns.

Code:
```python
def expectimax(self, board, depth, is_maximizing):
    if depth == 0 or self.is_terminal(board):  # Base case
        return self.evaluate(board)  # Evaluate the board at depth limit

    if is_maximizing:  # Player's turn (maximize)
        best_score = float('-inf')
        for move in self.game.get_possible_moves():  # Iterate over valid moves
            new_game = Game2048()
            new_game.board = board.copy()
            new_game.move(move)
            score = self.expectimax(new_game.board, depth - 1, False)  # Recurse for chance node
            best_score = max(best_score, score)  # Choose the max score
        return best_score
    else:  # Random tile placement (average expected score)
        empty_cells = [(i, j) for i in range(4) for j in range(4) if board[i, j] == 0]
        if not empty_cells:  # No empty cells, return evaluation
            return self.evaluate(board)
        score = 0
        for i, j in empty_cells:  # Iterate over all empty cells
            for tile_value, probability in [(2, 0.9), (4, 0.1)]:  # 90% for 2, 10% for 4
                new_board = board.copy()
                new_board[i, j] = tile_value
                score += probability * self.expectimax(new_board, depth - 1, True)  # Recurse for maximizing node
        return score
```

---

#### **Step 2: Handling the Player’s Turn (Maximizing Node)**
1. The AI simulates all possible moves:
   - `move(0)` for left, `move(1)` for up, `move(2)` for right, `move(3)` for down.
2. For each move, it calculates the **score of the resulting board** by recursively evaluating future moves and random events.
3. The AI selects the move with the highest score.

Key part of the code:
```python
if is_maximizing:
    best_score = float('-inf')
    for move in self.game.get_possible_moves():
        new_game = Game2048()
        new_game.board = board.copy()
        new_game.move(move)
        score = self.expectimax(new_game.board, depth - 1, False)
        best_score = max(best_score, score)
    return best_score
```

---

#### **Step 3: Handling Random Tile Placements (Chance Node)**
1. After the AI moves, a new tile (either `2` or `4`) spawns in a random empty cell.
2. The AI evaluates:
   - The **score** of placing a `2` in each empty cell (90% probability).
   - The **score** of placing a `4` in each empty cell (10% probability).
3. It calculates the **expected score** for all possible random tile placements and averages the results.

Key part of the code:
```python
else:
    empty_cells = [(i, j) for i in range(4) for j in range(4) if board[i, j] == 0]
    if not empty_cells:
        return self.evaluate(board)
    score = 0
    for i, j in empty_cells:
        for tile_value, probability in [(2, 0.9), (4, 0.1)]:
            new_board = board.copy()
            new_board[i, j] = tile_value
            score += probability * self.expectimax(new_board, depth - 1, True)
    return score
```

---

#### **Step 4: Evaluating the Board**
When the algorithm reaches the depth limit or a terminal state, it evaluates the board using a **heuristic evaluation function**.

In the provided code:
```python
def evaluate(self, board):
    return np.max(board) + np.sum(board)
```

This simple heuristic combines:
- The **maximum tile** (to encourage merging tiles).
- The **sum of all tiles** (to favor higher overall scores).

You could enhance this evaluation function to include:
- **Empty spaces**: Rewarding boards with more open tiles.
- **Smoothness**: Preferring boards where adjacent tiles have similar values.
- **Monotonicity**: Encouraging tiles to form a gradient (e.g., largest tiles in one corner).

---

### **4. Depth-Limited Search**
The algorithm uses a fixed depth (`depth=3` in this case) to balance:
- **Computation time**: A deeper search takes longer.
- **Accuracy**: A deeper search provides better decisions.

---

### **5. How Expectimax Relates to Game-Playing Principles**
1. **Search Tree**:
   - Represents all possible future states of the game.
   - Alternates between deterministic player moves and probabilistic random events.

2. **Evaluation**:
   - The algorithm estimates the quality of the board using heuristics when the search cannot go deeper.

3. **Optimal Decision-Making**:
   - By simulating multiple steps ahead, the AI selects the move with the highest expected score.

4. **Handling Uncertainty**:
   - Unlike deterministic algorithms (e.g., Minimax), Expectimax accounts for randomness by considering probabilities.

---

### **6. Key Advantages**
- Models randomness effectively, which is critical for games like 2048.
- Provides long-term planning by looking multiple steps ahead.

---

### **7. Simplified Summary**
1. The algorithm evaluates all possible moves (left, up, right, down).
2. It simulates the results of each move, considering both the player’s actions and the randomness of tile spawns.
3. It calculates the **expected score** for each move by:
   - Maximizing player’s outcomes.
   - Averaging random tile placements.
4. The AI selects the move with the highest expected score.

This approach ensures the AI makes informed decisions, balancing immediate rewards with long-term strategies. Let me know if you'd like further explanations or enhancements!